## Redes Neurais Profundas aplicadas a Retenção de Clientes - Data Processing and Preparation

### Extraindo dados do csv

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

# Algumas alterações foram feitas nesse dataset, com relação a padronização de dados
# Na coluna Gender (Genero) ficou assim: Male = 1. Female = 0
# Na coluna Geography (Pais) ficou assim: France: 0 Spain: 1 Germany: 2

# Carregamos os dados. Skiprows é para pular a primeira linha, com os títulos
raw_csv_data = np.loadtxt('ChurnModelling.csv',delimiter=',', skiprows = 1)

# Colocamos em uma variável a primeira e a última coluna do dataset, que são o ID e o target
unscaled_inputs_all = raw_csv_data[:,1:-1]

# Array contendo o alvo
targets_all = raw_csv_data[:,-1]

# Visualizando os dados carregados em um dataframe
df = pd.DataFrame(raw_csv_data)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,15634602.0,619.0,0.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0
1,15647311.0,608.0,1.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0
2,15619304.0,502.0,0.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0
3,15701354.0,699.0,0.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0
4,15737888.0,850.0,1.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0


### Balanceamento do Dataset

In [2]:
# Contar a quantidade de clientes que saiu Exited = 1
num_one_targets = int(np.sum(targets_all))

# Definir o contador para os clientes que não sairam
zero_targets_counter = 0

# Iremos criar um dataset balanceado
indices_to_remove = []

# Contar a quantidade de clientes que não sairam
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter = zero_targets_counter + 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Criamos duas novas variáveis, uma contendo as entradas e outra que contenha os alvos.
# Nós deletamos todos os índices que colocamos no array, no loop acima.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [3]:
unscaled_inputs_equal_priors = unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_equal_priors = targets_all = raw_csv_data[:,-1]

### Padronização das variáveis de input

In [4]:
# Esse é o único lugar onde usamos a funcionalidade sklearn. É uma linha simples de código, que padroniza as entradas.
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Alterando a ordem dos dados

##### Dadsos ordenados podem ser tendenciosos. Para tanto, fazemos o shuffle (embaralhamento) dos dados, para garantir que as amostras são, de fato, aleatórias.

In [5]:
# Quando os dados foram coletados, eles foram organizados por data
# Embaralhe os índices dos dados, para que os dados não sejam organizados de forma alguma quando os alimentarmos.
# Como os lotes serão enviados, queremos que os dados sejam distribuídos de forma tão aleatória quanto possível

shuffled_indices = np.arange(unscaled_inputs_equal_priors.shape[0])
np.random.shuffle(shuffled_indices)

# Use os índices embaralhados para embaralhar as entradas e destinos.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Dividimos o dataset em 3 partes: Treino, Validação e Teste

In [7]:
# Contamos a quantidade de amostras no dataset
samples_count = shuffled_inputs.shape[0]

# Quantidade de dados de treino
train_samples_count = int(0.8 * samples_count)
# Quantidade de dados de validação
validation_samples_count = int(0.1 * samples_count)
# Quantidade de dados de teste
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Variaveis para guardar os dados de treino
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Variaveis para guardar os dados de validação
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

# Variaveis para guardar os dados de teste - o que sobrou
test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

# Número de dados que são 1, o número total de amostras e a proporção para treinamento, validação e teste.
print('Quantidade de dados de treino: ', np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print('Quantidade de dados de validação: ', np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print('Quantidade de dados de teste: ', np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

Quantidade de dados de treino:  1618.0 8000 0.20225
Quantidade de dados de validação:  214.0 1000 0.214
Quantidade de dados de teste:  205.0 1000 0.205


### Salvamos os arquivos no formato .npz

In [9]:
np.savez('Churn_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Churn_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Churn_data_test', inputs=test_inputs, targets=test_targets)